#### Human Feedback 사용하기

Interrupt 걸어서 Feedback 받는 방식 2가지 있음.

1. compile 할때 `interrupt_before` 지정

```python 
graph = builder.compile(interrupt_before=["tools"],
                        checkpointer=ShortTermMemory)
```


2. 노드 내부에서 `NodeInterrupt` 발생

```python
raise NodeInterrupt(f"입력받은 문자열의 길이가 너무 깁니다. 최대 10자까지 입력해주세요.")
```

---

In [1]:
from modules.base import *

##### 1. 첫번째 방식 (interrupt_before)

In [2]:
@trace_function(enable_print=True, only_func_name=True)
def multiply(a: int, b: int) -> int:
    """Multiply a and b.

    Args:
        a: first int
        b: second int
    """
    return a * b

@trace_function(enable_print=True, only_func_name=True)
def add(a: int, b: int) -> int:
    """Adds a and b.
    Args:
        a: first int
        b: second int
    """
    return a + b

@trace_function(enable_print=True, only_func_name=True)
def divide(a: int, b: int) -> float:
    """Adds a and b.
    Args:
        a: first int
        b: second int
    """
    return a / b

tools = [add, multiply, divide]
llm_with_tools = llm.bind_tools(tools)

@trace_function(enable_print=True, only_func_name=True)
def node_assistant(state: MessagesState):
    system_message = SystemMessage(content="You are a helpful assistant tasked with performing arithmetic on a set of inputs.")
    return {"messages": [llm_with_tools.invoke([system_message] + state["messages"])]}
    
builder = StateGraph(MessagesState)
builder.add_node("node_assistant", node_assistant)
builder.add_node("tools", ToolNode(tools))
builder.add_edge(START, "node_assistant")
builder.add_conditional_edges("node_assistant",tools_condition)
builder.add_edge("tools", "node_assistant")
config = {"configurable": {"thread_id": "1"}}

breakpoint 간단 예제 - Tool Node 에 Breakpoint 설정

- 함수들에도 `trace_function` 데코레이터를 붙였지만, 통과하지 않음.

- state의 messages 값의 마지막도 tool call 요청문

- Graph 이어서 실행시, 사용자에게 입력받음

In [3]:
ShortTermMemory = MemorySaver()
graph_with_interrupt_tools = builder.compile(interrupt_before=["tools"], # tools 노드 전에 멈춤
                                             checkpointer=ShortTermMemory)

In [4]:
graph_with_interrupt_tools.invoke({"messages": "Multiply 2 and 3"}, config)
graph_with_interrupt_tools.get_state(config).values


🚀 Passing Through [node_assistant] ..


{'messages': [HumanMessage(content='Multiply 2 and 3', additional_kwargs={}, response_metadata={}, id='fa74446b-ff81-4116-87c3-d49d28e6b98e'),
  AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_mkVdFzg22CeMhet3fXo6Stoq', 'function': {'arguments': '{"a":2,"b":3}', 'name': 'multiply'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 18, 'prompt_tokens': 143, 'total_tokens': 161, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': 'fp_b7d65f1a5b', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-905af210-3c09-4411-a588-1001e5ed6b96-0', tool_calls=[{'name': 'multiply', 'args': {'a': 2, 'b': 3}, 'id': 'call_mkVdFzg22CeMhet3fXo6Stoq', 'type': 'tool_call'}], usage_metadata={'input_tokens': 143, 'output_tokens'

In [5]:
user_approval = input("Do you want to call the tool? (yes/no): ")
if user_approval.lower() == "yes":
    graph_with_interrupt_tools.invoke(None, config) # yes 일 경우 tool 노드 호출


🚀 Passing Through [multiply] ..

🚀 Passing Through [node_assistant] ..


In [6]:
graph_with_interrupt_tools.get_state(config).values

{'messages': [HumanMessage(content='Multiply 2 and 3', additional_kwargs={}, response_metadata={}, id='fa74446b-ff81-4116-87c3-d49d28e6b98e'),
  AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_mkVdFzg22CeMhet3fXo6Stoq', 'function': {'arguments': '{"a":2,"b":3}', 'name': 'multiply'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 18, 'prompt_tokens': 143, 'total_tokens': 161, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': 'fp_b7d65f1a5b', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-905af210-3c09-4411-a588-1001e5ed6b96-0', tool_calls=[{'name': 'multiply', 'args': {'a': 2, 'b': 3}, 'id': 'call_mkVdFzg22CeMhet3fXo6Stoq', 'type': 'tool_call'}], usage_metadata={'input_tokens': 143, 'output_tokens'

Humanfeedback 간단 예제 - Assistant Node 에 Breakpoint 설정

- state의 messages 값의 마지막이 사용자 요청문

- `update_state`를 통해 사용자 요청문 재입력 가능

In [7]:
ShortTermMemory = MemorySaver()
graph_with_interrupt_assitant = builder.compile(interrupt_before=["node_assistant"], # node_assistant 노드 전에 멈춤
                                                checkpointer=ShortTermMemory)

In [8]:
graph_with_interrupt_assitant.invoke({"messages": "2 곱하기 3은?"}, config)
graph_with_interrupt_assitant.get_state(config).values

{'messages': [HumanMessage(content='2 곱하기 3은?', additional_kwargs={}, response_metadata={}, id='f0afa064-a46b-419e-853d-5a67c14712d0')]}

In [9]:
graph_with_interrupt_assitant.update_state(config,
                                           {"messages": [HumanMessage(content="아! 잘못말헀다. 3 곱하기 3은?")]})
graph_with_interrupt_assitant.invoke(None, config) # 실제로 3 x 3 계산 수행
graph_with_interrupt_assitant.get_state(config).values


🚀 Passing Through [node_assistant] ..

🚀 Passing Through [multiply] ..


{'messages': [HumanMessage(content='2 곱하기 3은?', additional_kwargs={}, response_metadata={}, id='f0afa064-a46b-419e-853d-5a67c14712d0'),
  HumanMessage(content='아! 잘못말헀다. 3 곱하기 3은?', additional_kwargs={}, response_metadata={}, id='d11d7b3a-8602-459a-b239-f504022ac91c'),
  AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_9kAgyb18v2jZb8ygXe3NCW9A', 'function': {'arguments': '{"a":3,"b":3}', 'name': 'multiply'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 18, 'prompt_tokens': 167, 'total_tokens': 185, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': 'fp_b7d65f1a5b', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-6c5d90bf-ddba-46d0-8992-be72d6a37170-0', tool_calls=[{'name': 'multiply', 'args': {'a':

##### 2. Node 내부에서 NodeInterrupt 발생

In [14]:
from langgraph.errors import NodeInterrupt

@trace_function(enable_print=True, only_func_name=True)
def step_1(state:MessagesState) -> MessagesState:
    return state

@trace_function(enable_print=True, only_func_name=True)
def step_2(state:MessagesState) -> MessagesState:
    if len(state['messages'][-1].content) > 5:
        print(f"Received input that is longer than 5 characters: {state['messages'][-1].content}")
        raise NodeInterrupt(f"Received input that is longer than 5 characters: {state['messages'][-1].content}")
    return state

@trace_function(enable_print=True, only_func_name=True)
def step_3(state:MessagesState) -> MessagesState:
    return state

builder = StateGraph(MessagesState)
builder.add_node("step_1", step_1)
builder.add_node("step_2", step_2)
builder.add_node("step_3", step_3)
builder.add_edge(START, "step_1")
builder.add_edge("step_1", "step_2")
builder.add_edge("step_2", "step_3")
builder.add_edge("step_3", END)
memory = MemorySaver()
graph_with_node_interrupt = builder.compile(checkpointer=memory)

In [15]:
graph_with_node_interrupt.invoke({"messages": "안녕하세요, 저는 창우라고 합니다."}, config)
graph_with_node_interrupt.get_state(config).values


🚀 Passing Through [step_1] ..

🚀 Passing Through [step_2] ..
Received input that is longer than 5 characters: 안녕하세요, 저는 창우라고 합니다.


{'messages': [HumanMessage(content='안녕하세요, 저는 창우라고 합니다.', additional_kwargs={}, response_metadata={}, id='3b81c80f-082b-4fff-9c1d-f6a3c6ed782d')]}

In [16]:
graph_with_node_interrupt.update_state(config, {"messages": [HumanMessage(content="안녕")]})
graph_with_node_interrupt.invoke(None, config)
graph_with_node_interrupt.get_state(config).values


🚀 Passing Through [step_2] ..

🚀 Passing Through [step_3] ..


{'messages': [HumanMessage(content='안녕하세요, 저는 창우라고 합니다.', additional_kwargs={}, response_metadata={}, id='3b81c80f-082b-4fff-9c1d-f6a3c6ed782d'),
  HumanMessage(content='안녕', additional_kwargs={}, response_metadata={}, id='03c51dd0-7d02-4b29-b6f2-6063d573870c')]}